# Data Labelling

In [1]:
!pip install tensorflow

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
# from ml_logic.params import *
from tensorflow.keras import optimizers,Model, Sequential, layers, metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import os
# from google.cloud import storage
# from google.oauth2 import service_account
# import tqdm
import numpy as np

2023-03-10 11:13:15.739526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 11:13:16.321422: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 11:13:16.321440: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-10 11:13:16.387458: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-10 11:13:17.621885: W tensorflow/stream_executor/platform/de

### Import the train, test, val directory paths which will be used by the image_dataset_from_directory function

In [2]:
train_directory = '/root/code/FlorenceBoutin/raw_data/Gi_disease_data/train'
test_directory = '/root/code/FlorenceBoutin/raw_data/Gi_disease_data/test'
val_directory = '/root/code/FlorenceBoutin/raw_data/Gi_disease_data/val'



In [3]:
val_directory

'/root/code/FlorenceBoutin/raw_data/Gi_disease_data/val'

In [4]:
def load_images(path):
        """
        Enter a path to load images from.
        """
        datagen = ImageDataGenerator(rescale = 1. / 255)
        images = datagen.flow_from_directory(path,
                                             target_size = (224,224),
                                             color_mode = "rgb",
                                             batch_size = 32,
                                             class_mode = "categorical")

        return images


In [5]:
train_dataset = load_images(train_directory)
val_dataset = load_images(val_directory)
test_dataset = load_images(test_directory)

Found 3200 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 800 images belonging to 4 classes.


In [6]:
x = 1
x

1

In [7]:
def initialize_baseline_model(input_shape=(224, 224, 3)):
    """
    Initialize the Neural Network with random weights
    """

    # Create instance of to model
    model = Sequential()

    # Add Convolution layers + Pooling and Dropout layers to limit overfitting.
    model.add(layers.Conv2D(64, kernel_size=(3,3), input_shape=input_shape, activation='relu', padding='same'))
    model.add(layers.MaxPool2D(pool_size=(3,3)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(32, kernel_size=(2,2), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Conv2D(16, kernel_size=(2,2), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    # Flatten and Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(4, activation='softmax'))

    print("✅ baseline model initialized")

    return model

In [8]:
def compile_baseline_model(model, learning_rate=0.001):
    """
    Compile the Neural Network
    """
    recall = metrics.Recall()
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy',
               optimizer=optimizer,
               metrics=[recall, 'accuracy'])

    print("✅ model compiled")
    return model

In [9]:
def train_baseline_model(model,
                         train_data,
                         validation_data,
                         patience=5) :
    """
    Fit model and return a the tuple (fitted_model, history)
    """

    es = EarlyStopping(
        monitor="val_loss",
        patience=patience,
        restore_best_weights=True,
        verbose=0
    )

    history = model.fit(train_data,
        validation_data=validation_data,
        epochs=30,
        callbacks=[es],
        verbose=1)

    print(f"✅ model trained with accuracy of {round(np.max(history.history['val_accuracy']), 2)} and a recall of {round(np.max(history.history['val_recall']), 2)}.")

    return model, history

In [10]:
model = initialize_baseline_model(input_shape=(224, 224, 3))
model

✅ baseline model initialized


2023-03-10 11:13:34.651254: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-10 11:13:34.652003: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-10 11:13:34.652040: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-FR6NK48): /proc/driver/nvidia/version does not exist
2023-03-10 11:13:34.653423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
model = compile_baseline_model(model, learning_rate=0.001)
model

✅ model compiled


In [14]:
model, history = train_baseline_model(model,
                         train_dataset,
                         val_dataset,
                         patience=5)

Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/root/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/root/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/root/.pyenv/versions/lewagon/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/root/.pyenv/versions/lewagon/lib/python3.10/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/root/.pyenv/versions/lewagon/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/root/.pyenv/versions/lewagon/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loo

In [35]:
print(history)